In [21]:
import warnings
warnings.filterwarnings('ignore')

데이터는 mysql에 저장해두며 필요한 작업마다 mysql에서 호출해서 불러옵니다.

In [22]:
# mysql에 저장한 데이터를 불러오기

import mysql.connector
import pandas as pd
import numpy as np

import mysql.connector
import pandas as pd

# MySQL 연결 설정
cnx = mysql.connector.connect(
    host='',
    user='',
    password='',
    database='trainDB(reverse y)'
)

# 쿼리 생성
query_ball = "SELECT * FROM ball_markup_1"
query_event = "SELECT * FROM events_markup_1"

# DataFrame으로 읽기
ball_df = pd.read_sql(query_ball, con=cnx)
events_df = pd.read_sql(query_event, con=cnx)

# 연결 종료 (선택적)
cnx.close()

In [23]:
# mysql에서 불러올 때 자동으로 생성되는 열 제거
ball_df = ball_df.drop(columns='level_0')
ball_df.head()

,index,frame,x,y
0,0,14,506,221
1,1,15,523,215
2,2,16,537,210
3,3,17,552,203
4,4,18,566,200


**ball_df**에서 **frame**은 동영상의 프레임 번호, **x**,**y**는 공의 좌표입니다.

In [24]:
# mysql에서 불러올 때 자동으로 생성되는 열 제거
events_df = events_df.drop(columns='level_0')
events_df.head()

,index,frame,event
0,0,18,bounce
1,1,33,net
2,2,44,bounce
3,3,58,empty_event
4,4,98,net


**event_df**에서 **frame**은 동영상의 프레임 번호, **event**는 발생한 이벤트입니다.

* **bounce**: 공이 테이블 위에 튕김
* **net**: 공이 네트를 지나감 (net에 맞는게 아님)


In [25]:
# 프레임값이 str이라 int로 변경
ball_df['frame'] = ball_df['frame'].astype(int)
events_df['frame'] = events_df['frame'].astype(int)

In [26]:
# 프레임 오름차순 확인
is_increasing_1 = ball_df['frame'].is_monotonic_increasing
is_increasing_2 = events_df['frame'].is_monotonic_increasing
print(is_increasing_1, is_increasing_2)

True True


두 데이터프레임으로 추가로 생성할 수 있는 feature들입니다.
* **player**: 어느 쪽 선수가 친 공인지 구분
* **velocity**: 해당 좌표에서 공의 순간속도 (각도 문제가 있으나 카메라 각도상 각도가 크지 않아 일단은 포함)
* ~~**angle**: 카메라가 바라본 방향에서의 공의 각도를 구해도 너무 부정확해 활용할 수 없다고 판단하여 제외~~
* **avg_velocity**: 특정 구간에서의 공의 평균 속도 (추후 구간 별로 나누고 해당 구간 별 평균속도 구할 예정)

##### **player**열 생성

**ball_df** 데이터에서 공이 날아온 벡터 방향을 토대로 어느쪽 선수가 친 공인지 파악할 수 있습니다.\
\
예를 들어, 공의 벡터의 이동이 오른쪽에서 왼쪽이면 오른쪽 선수가 쳤다고 볼 수 있습니다.

<추후 발견된 문제점>\
\
공이 네트를 맞는 등의 이벤트로 공의 이동 방향이 바뀌고 이를 인식해 플레이어가 바뀌는 경우가 발견되었습니다. 이는 추후 간접적으로 해결합니다.

In [27]:
# Player 열을 None값으로 설정
ball_df['player'] = None

# 반복을 통해 x,y 좌표를 고려해 공의 방향에 따라 선수 배정
for i in range(1, len(ball_df)):
    if ball_df.loc[i, 'frame'] - ball_df.loc[i-1, 'frame'] == 1:  # 동영상 프레임이 연속적인지 체크
        delta_x = ball_df.loc[i, 'x'] - ball_df.loc[i-1, 'x']
        delta_y = ball_df.loc[i, 'y'] - ball_df.loc[i-1, 'y']
        
        # 공의 벡터 방향에 따라 플레이어를 결정
        if delta_x > 0 and delta_y > 0:  # 공이 오른쪽 아래로 이동
            ball_df.loc[i, 'player'] = 'player_A'
        elif delta_x < 0 and delta_y < 0:  # 공이 왼쪽 및 위쪽으로 이동
            ball_df.loc[i, 'player'] = 'player_B'
        elif delta_x > 0:  # 공이 오른쪽으로만 이동
            ball_df.loc[i, 'player'] = 'player_A'
        else:  # 공이 왼쪽으로만 이동
            ball_df.loc[i, 'player'] = 'player_B'

# [-1, -1] 제거, [-1. -1]은 공이 나간 경우를 의미하므로 제거
ball_df = ball_df[ball_df['x'] != -1]
ball_df.head()

,index,frame,x,y,player
0,0,14,506,221,None
1,1,15,523,215,player_A
2,2,16,537,210,player_A
3,3,17,552,203,player_A
4,4,18,566,200,player_A


0행의 경우 앞의 참고할 데이터가 없으니 None으로 나타납니다.

In [28]:
# 결측치 전부 삭제
ball_df = ball_df.dropna()

In [29]:
ball_df = ball_df.drop(columns='index')

이제 구간 별로 sequence를 나눠줄 것입니다.\
\
제공받은 공의 좌표 데이터는 동영상의 전체를 나타내지 않고 부분부분 잘려있습니다.\
\
이를 구간 별로 나누고 유의미한 구간만 남길 것입니다.\
\
영상 프레임 차이가 10 이내인 구간들은 이어지는 구간이라 판단하고, 10이 넘어가면 완전 별개의 구간이라고 판단합니다.

In [30]:
# 인덱스 재설정
ball_df.reset_index(drop=True, inplace=True)

# sequence 열을 다시 초기화
ball_df['sequence'] = 0

# sequence 할당을 위한 로직 적용
current_sequence = 1
for i in range(len(ball_df)):
    ball_df.at[i, 'sequence'] = current_sequence
    # frame 차이가 10보다 크면 완전히 다른 구간과 장면이라고 판단합니다. 10 이하로는 같은 구간이라도 프레임이 잠깐 끊긴 경우일 수 있습니다.
    if i == len(ball_df) - 1 or ball_df.at[i + 1, 'frame'] - ball_df.at[i, 'frame'] > 10:
        current_sequence += 1

# 열을 정렬하여 sequence를 앞으로 가져오기
column_order = ['sequence'] + [col for col in ball_df.columns if col != 'sequence']
ball_df = ball_df[column_order]

# 문자열을 int로 변경
ball_df['frame'] = ball_df['frame'].astype(int)
ball_df['x'] = ball_df['x'].astype(int)
ball_df['y'] = ball_df['y'].astype(int)

ball_df.head()

,sequence,frame,x,y,player
0,1,15,523,215,player_A
1,1,16,537,210,player_A
2,1,17,552,203,player_A
3,1,18,566,200,player_A
4,1,19,578,205,player_A


총 237 sequence까지 생성되었습니다.

In [31]:
ball_df['sequence'].value_counts

<bound method IndexOpsMixin.value_counts of 0         1
1         1
2         1
3         1
4         1
       ... 
6246    237
6247    237
6248    237
6249    237
6250    237
Name: sequence, Length: 6251, dtype: int64>

* 앞에서 언급한 player 문제를 임시방편으로 해결합니다. \
(임시방편이 아닌 확실한 로직으로 player 구분을 하려 했으니 추가 고려사항이 필요해 제공받은 공의 x,y좌표만으로는 불가능)

* 같은 sequence 내 **두번째** 플레이어로 모두 통일해주는 것으로 어느 정도 임시방편 가능합니다.
  (첫번째로 할 경우 시작지점 순간적으로 공의 방향이 전환되는 경우가 있어서 중간에 잘못된 sequence들이 생깁니다.)
    * sequecen 내에서는 공의 움직임이 이어지는 같은 구간입니다. 그리고 이 구간 안에서는 확인 결과 선수들끼리 공을 주고 받는 장면이 아닌 한 선수의 스매싱 과정만 해당됨을 알 수 있었습니다. 
    * 그렇기에 제일 처음 나오는 player로 고정해두고 그 뒤로는 공의 방향이 바뀌어도 player가 변경되지 않도록 합니다.

In [32]:
# 'sequence'로 그룹화하고 각 그룹의 두 번째 'player' 값을 사용하여 그룹의 모든 행을 업데이트합니다.
# ball_df['player'] = ball_df.groupby('sequence')['player'].transform('second') 
# 첫번째로 하면 잘목된 sequence들이 생김)

def second_element(x):
    if len(x) > 1:
        return x.iloc[1]
    else:
        return x.iloc[0]  # 또는 None, 만약 그룹에 요소가 하나밖에 없는 경우

ball_df['player'] = ball_df.groupby('sequence')['player'].transform(second_element)

**ball_df**를 이용하여 구간 별로 sequence를 구분한 **sequence_df**을 생성해줍니다.

In [33]:
# 'sequence'로 그룹화
grouped = ball_df.groupby('sequence')

# 각 그룹에서 'frame'의 최솟값과 최댓값에 해당하는 행만 선택
selected_rows = []
for name, group in grouped:
    min_idx = group['frame'].min()
    max_idx = group['frame'].max()
    
    # 최솟값과 최댓값이 같지 않은 경우만 선택
    if min_idx != max_idx:
        selected_rows.append(group[group['frame'] == min_idx])
        selected_rows.append(group[group['frame'] == max_idx])

# 선택된 행들로 새로운 데이터프레임 생성
sequence_df = pd.concat(selected_rows)
sequence_df = sequence_df.sort_values('frame')
sequence_df.head()

,sequence,frame,x,y,player
0,1,15,523,215,player_A
40,1,56,109,172,player_A
41,2,95,836,446,player_A
56,2,110,1057,400,player_A
57,3,135,1298,218,player_A


이제 frame와 x,y를 두 구간으로 나누어줍니다.\
\
구간 별로 frame1 ~ frame2 / x1 ~ x2 / y1 ~ y2 이런 식으로

In [34]:
# 'x'와 'y'의 이름을 각각 'x1'과 'y1'으로 변경
sequence_df = sequence_df.rename(columns={"x": "x1", "y": "y1"})

# 'x2'와 'y2' 열을 생성
sequence_df["x2"] = sequence_df.groupby("sequence")["x1"].shift(-1)
sequence_df["y2"] = sequence_df.groupby("sequence")["y1"].shift(-1)

# 'frame' 열의 이름을 'frame_1'로 변경
sequence_df["frame_1"] = sequence_df["frame"]

# 'frame_2' 열 생성
sequence_df["frame_2"] = sequence_df.groupby("sequence")["frame"].shift(-1)

# 'avg_velocity'의 고유값을 1개만 남기기 위해 두 번째 행 삭제
sequence_df = sequence_df.groupby("sequence").first().reset_index()

# 'x2'와 'y2' 열을 정수로 변환
sequence_df['x2'] = sequence_df['x2'].astype(int)
sequence_df['y2'] = sequence_df['y2'].astype(int)

# 'frame_2'를 정수로 변환
sequence_df['frame_2'] = sequence_df['frame_2'].astype(int)

# 프레임 차이
sequence_df['diff_frame'] = sequence_df['frame_2'] - sequence_df['frame_1']

# 열 재정렬
ordered_columns = ['sequence','frame_1', 'frame_2', 'diff_frame', 'x1', 'y1', 'x2', 'y2', 'player']
sequence_df = sequence_df[ordered_columns]

# 인덱스 초기화
sequence_df = sequence_df.reset_index(drop=True)

sequence_df.head()

,sequence,frame_1,frame_2,diff_frame,x1,y1,x2,y2,player
0,1,15,56,41,523,215,109,172,player_A
1,2,95,110,15,836,446,1057,400,player_A
2,3,135,150,15,1298,218,1451,168,player_A
3,4,2189,2223,34,1407,124,704,254,player_B
4,5,2236,2251,15,558,224,582,269,player_B


#### net와 bounce가 모두 포함된 sequence만 남겨줍니다.
- sequnece 별로 일정 프레임 이하의 sequnce를 제외하는 방법도 적용했는데 기준이 애매하여 결과물이 불안정했습니다.
- 하지만 net와 bounce가 모두 포함된 sequence만 남기면 해결됩니다. 공은 net를 넘어가서 bounce를 발생시키기 때문입니다.
  
  
<공이 그대로 아웃되는 경우 net를 넘어가고 bounce는 발생하지 않지만 제공받은 데이터셋에서 공의 움직임이 공이 아웃될 때까지 쭉 이어지는 경우가 거의 없었기 때문에 아쉽게도 공이 아웃되는 경우는 고려사항에서 제외했습니다. 만약 이게 고려가 가능했으면 플레이어 별 아웃된 공에 대한 분석도 가능했을 겁니다.>

In [35]:
# events_df에서 'bounce' 이벤트와 'net' 이벤트가 있는 frame 찾기
bounce_frames = events_df[events_df['event'] == 'bounce']['frame']
net_frames = events_df[events_df['event'] == 'net']['frame']

# ball_df에서 해당 frame을 갖는 sequence를 찾기
bounce_sequences = ball_df[ball_df['frame'].isin(bounce_frames)]['sequence']
net_sequences = ball_df[ball_df['frame'].isin(net_frames)]['sequence']

# 'bounce' 이벤트와 'net' 이벤트가 모두 포함된 sequence를 찾습니다
valid_sequences = set(bounce_sequences) & set(net_sequences)

# ball_df에서 해당 sequence을 가진 행만 남깁니다.
ball_df = ball_df[ball_df['sequence'].isin(valid_sequences)]

# 연속된 순서로 sequence 번호를 재설정 합니다. (중간에 빈 sequence가 생기니 1부터 다시 순서대로 정렬)
sequence_mapping = {seq: i+1 for i, seq in enumerate(sorted(valid_sequences))}
ball_df['sequence'] = ball_df['sequence'].map(sequence_mapping)

ball_df.head()

,sequence,frame,x,y,player
0,1,15,523,215,player_A
1,1,16,537,210,player_A
2,1,17,552,203,player_A
3,1,18,566,200,player_A
4,1,19,578,205,player_A


**sequence_df**에 대해서도 똑같이 적용합니다. \
\
적용된 ball_df에서 앞에서 sequence_df을 구한 과정을 한번 더 반복하면 됩니다.

In [36]:
# 'sequence'로 그룹화
grouped = ball_df.groupby('sequence')

# 각 그룹에서 'frame'의 최솟값과 최댓값에 해당하는 행만 선택
selected_rows = []
for name, group in grouped:
    min_idx = group['frame'].min()
    max_idx = group['frame'].max()
    
    # 최솟값과 최댓값이 같지 않은 경우만 선택
    if min_idx != max_idx:
        selected_rows.append(group[group['frame'] == min_idx])
        selected_rows.append(group[group['frame'] == max_idx])

# 선택된 행들로 새로운 데이터프레임 생성
sequence_df = pd.concat(selected_rows)
sequence_df = sequence_df.sort_values('frame')

# ---------------------------------------------
# 'x'와 'y'의 이름을 각각 'x1'과 'y1'으로 변경
sequence_df = sequence_df.rename(columns={"x": "x1", "y": "y1"})

# 'x2'와 'y2' 열을 생성
sequence_df["x2"] = sequence_df.groupby("sequence")["x1"].shift(-1)
sequence_df["y2"] = sequence_df.groupby("sequence")["y1"].shift(-1)

# 'frame' 열의 이름을 'frame_1'로 변경
sequence_df["frame_1"] = sequence_df["frame"]

# 'frame_2' 열 생성
sequence_df["frame_2"] = sequence_df.groupby("sequence")["frame"].shift(-1)

# 'avg_velocity'의 고유값을 1개만 남기기 위해 두 번째 행 삭제
sequence_df = sequence_df.groupby("sequence").first().reset_index()

# 'x2'와 'y2' 열을 정수로 변환
sequence_df['x2'] = sequence_df['x2'].astype(int)
sequence_df['y2'] = sequence_df['y2'].astype(int)

# 'frame_2'를 정수로 변환
sequence_df['frame_2'] = sequence_df['frame_2'].astype(int)

# 프레임 차이
sequence_df['diff_frame'] = sequence_df['frame_2'] - sequence_df['frame_1']

# 열 재정렬
ordered_columns = ['sequence','frame_1', 'frame_2', 'diff_frame', 'x1', 'y1', 'x2', 'y2', 'player']
sequence_df = sequence_df[ordered_columns]

# 인덱스 초기화
sequence_df = sequence_df.reset_index(drop=True)

sequence_df.head()

,sequence,frame_1,frame_2,diff_frame,x1,y1,x2,y2,player
0,1,15,56,41,523,215,109,172,player_A
1,2,2189,2223,34,1407,124,704,254,player_B
2,3,2267,2299,32,848,262,1475,177,player_A
3,4,9240,9299,59,1269,163,612,244,player_B
4,5,9340,9369,29,874,254,1137,208,player_A


이제 **events_df**에 있는 특정 이벤트가 발생한 좌표들을 **ball_df**와 결합하여 어느 좌표에서 누가 공을 친 상황에서 특정 이벤트들이 발생했는지를 나타내는 데이터프레임을 생성합니다.

In [37]:
# ball_df와 events_df를 frame의 교집합 반환한 merged_df 생성

# events_df에 존재하는 프레임 값을 기준으로 ball_df를 필터링
filtered_df = ball_df[ball_df['frame'].isin(events_df['frame'])]

# 'frame' 열을 기준으로 events_df에서 filter_df로 'event' 열 병합
merged_df = pd.merge(filtered_df, events_df[['frame', 'event']], on='frame', how='left')

# 열을 재정렬하여 'y'와 'player' 사이에 'event'를 배치
merged_df = merged_df[['frame', 'x', 'y', 'event', 'player']]

# bounce,net 외 empty_event라는 이벤트도 있는데 이는 분석에 필요가 없으니 제외
events_df = merged_df[merged_df['event'] != 'empty_event']

# events_df에서 ball_df에 포함되지 않은 frame은 삭제 (ball_df에서 프레임 일부 제외해줬으므로)
events_df = events_df[events_df['frame'].isin(ball_df['frame'])]

events_df.head()

,frame,x,y,event,player
0,18,566,200,bounce,player_A
1,33,897,228,net,player_A
2,44,500,138,bounce,player_A
3,2192,1305,110,bounce,player_B
4,2211,889,240,net,player_B


In [38]:
# ball_df.to_csv('ball_df.csv')
# sequence_df.to_csv('sequence_df.csv')
# events_df.to_csv('events_df.csv')

In [39]:
# mysql로 보내기

from sqlalchemy import create_engine
 
db_connection_path = ''
db_connection = create_engine(db_connection_path)

conn = db_connection.connect()

ball_df.to_sql('ball_df', con = db_connection, if_exists= 'replace')
sequence_df.to_sql('sequence_df', con = db_connection, if_exists= 'replace')
events_df.to_sql('events_df', con = db_connection, if_exists= 'replace')

335